In [1]:
! pip install transformers

     |████████████████████████████████| 2.9 MB 9.5 MB/s 
     |████████████████████████████████| 895 kB 68.0 MB/s 
     |████████████████████████████████| 3.3 MB 56.3 MB/s 
     |████████████████████████████████| 636 kB 57.4 MB/s 
     |████████████████████████████████| 52 kB 2.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
from google.colab import drive
drive.mount('/drive')


Mounted at /drive


In [4]:
drive_dir = '/drive/MyDrive'
data_dir = drive_dir + '/nlp_nature'

In [5]:
import pandas as pd
conclusion_df=pd.read_csv(data_dir + '/jco_conclusion_df_bevacizumab_29092021.csv')

In [6]:
conclusion_df.head()

,Unnamed: 0,link,title,journal,authors,type,field,article,conclusion
0,0,https://ascopubs.org/doi/abs/10.1200/JCO.20.01225,\n\n \n \n\n ...,Journal of Clinical Oncology,"Chiara Cremolini,Search for articles by this a...",ORIGINAL REPORTS,Gastrointestinal Cancer,\n##Abstract##PURPOSE##A proper estimation of ...,FOLFOXIRI + bevacizumab significantly and mean...
1,1,https://ascopubs.org/doi/abs/10.1200/JOP.2015....,\n\n \n \n\n ...,Journal of Oncology Practice,"Stacie B. Dusetzina,Search for articles by thi...",Health Policy,Original Contributions,\n##Abstract##Section:##Choose##Top of page##A...,FOLFOXIRI + bevacizumab significantly and mean...
2,2,https://ascopubs.org/doi/abs/10.1200/JCO.2015....,\n\n \n \n\n ...,Journal of Clinical Oncology,"Bradley J. Monk,Search for articles by this au...",ORIGINAL REPORTS,Gynecologic Cancer,\n##Abstract##Section:##Choose##Top of page##A...,"On the basis of the PFS, overall response rate..."
3,3,https://ascopubs.org/doi/abs/10.1200/JCO.19.03158,\n\n \n \n\n ...,Journal of Clinical Oncology,"Martin Reck,Search for articles by this author...",ORIGINAL REPORTS,Thoracic Oncology,\n##Abstract##Section:##Choose##Top of page##A...,ABCP seems tolerable and manageable versus ACP...
4,4,https://ascopubs.org/doi/abs/10.1200/JCO.2014....,\n\n \n \n\n ...,Journal of Clinical Oncology,"Miguel Martín,Search for articles by this auth...",ORIGINAL REPORTS,Breast Cancer,\n##Abstract##Section:##Choose##Top of page##A...,The addition of bevacizumab to ET in first-l...


In [10]:
import numpy as np
import torch

## Load the transformer


This model is a fine-tuned checkpoint of RoBERTa-large (Liu et al. 2019). It enables reliable binary sentiment analysis for various types of English-language text. For each instance, it predicts either positive (1) or negative (0) sentiment. The model was fine-tuned and evaluated on 15 data sets from diverse text sources to enhance generalization across different types of texts (reviews, tweets, etc.). Consequently, it outperforms models trained on only one type of text (e.g., movie reviews from the popular SST-2 benchmark) when used on new data as shown below.

## Performance : 

To evaluate the performance of our general-purpose sentiment analysis model, we set aside an evaluation set from each data set, which was not used for training. On average, our model outperforms a DistilBERT-based model (which is solely fine-tuned on the popular SST-2 data set) by more than 15 percentage points (78.1 vs. 93.2 percent, see table below). As a robustness check, we evaluate the model in a leave-one-out manner (training on 14 data sets, evaluating on the one left out), which decreases model performance by only about 3 percentage points on average and underscores its generalizability. Model performance is given as evaluation set accuracy in percent. Dataset DistilBERT SST-2 This model McAuley and Leskovec (2013) (Reviews) 84.7 98.0 McAuley and Leskovec (2013) (Review Titles) 65.5 87.0 Yelp Academic Dataset 84.8 96.5 Maas et al. (2011) 80.6 96.0 Kaggle 87.2 96.0 Pang and Lee (2005) 89.7 91.0 Nakov et al. (2013) 70.1 88.5 Shamma (2009) 76.0 87.0 Blitzer et al. (2007) (Books) 83.0 92.5 Blitzer et al. (2007) (DVDs) 84.5 92.5 Blitzer et al. (2007) (Electronics) 74.5 95.0 Blitzer et al. (2007) (Kitchen devices) 80.0 98.5 Pang et al. (2002) 73.5 95.5 Speriosu et al. (2011) 71.5 85.5 Hartmann et al. (2019) 65.5 98.0 Average 78.1 93.2

In [11]:
# Test the GPU
torch.cuda.is_available()

True

In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [14]:
from transformers import AutoTokenizer,  AutoModelForSequenceClassification

model_name = "ProsusAI/finbert"

tokenizer = AutoTokenizer.from_pretrained(model_name)
classifier = AutoModelForSequenceClassification.from_pretrained(model_name)


Downloading:   0%|          | 0.00/252 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/758 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

In [15]:
l_test = ['The worst firm I\' ve ever quoted.', 'Apple is doing tremendously well this year.', 'Everybody is sitting.']

# Use the datas

In [17]:
def assign_device(token_outputs) :
   token_tensor = token_outputs['input_ids'].to(device)
   token_type_ids = token_outputs['token_type_ids'].to(device)
   attention_mask = token_outputs['attention_mask'].to(device)

   output = {'input_ids' : token_tensor, 
          'token_type_ids' : token_type_ids, 
          'attention_mask' : attention_mask}

   return output

In [18]:


# Tokenize sentence list 
input_tokens = assign_device(tokenizer(l_test , truncation=True, padding=True, return_tensors="pt"))

classifier_device = classifier.to(device)
# Model apply to get the missing logits
outputs = classifier_device(**input_tokens)

print(input_tokens)
print(outputs)



{'input_ids': tensor([[  101,  1996,  5409,  3813,  1045,  1005,  2310,  2412,  9339,  1012,
           102],
        [  101,  6207,  2003,  2725, 14388,  2135,  2092,  2023,  2095,  1012,
           102],
        [  101,  7955,  2003,  3564,  1012,   102,     0,     0,     0,     0,
             0]], device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]], device='cuda:0')}
SequenceClassifierOutput(loss=None, logits=tensor([[-1.8616,  2.6339,  0.0093],
        [ 2.0206, -2.1929, -1.0047],
        [-1.0452, -0.5722,  2.0732]], device='cuda:0', grad_fn=<AddmmBackward>), hidden_states=None, attentions=None)


In [22]:
#Transform the logits to probabilities
softmax=torch.nn.Softmax(dim=1).to(device)

#Transform the logits tensor to numpy, add cpu()
logits = outputs.logits.cpu().detach().numpy()
probs=softmax(outputs.logits).cpu().detach().numpy()
print(logits)
print(probs)

[[-1.8616208   2.6338508   0.00933403]
 [ 2.0205882  -2.192863   -1.0047042 ]
 [-1.0452498  -0.57215476  2.073244  ]]
[[0.01029814 0.9228207  0.06688122]
 [0.9404341  0.01391392 0.04565208]
 [0.03965538 0.06364508 0.89669955]]
